In [34]:
import numpy as np
import onnxruntime as ort
import onnx
import onnx_graphsurgeon as gs
from pathlib import Path
from rich import print

In [49]:
inputs="reference_line_valid_mask"
  
# path=Path("D:/Learn DL/Emil-Net/src/isolated_emil_net.onnx")

path=Path("D:/Learn DL/Emil-Net/src/ak-isolated.onnx")

input_value = np.random.rand(1,1,99).astype(np.float32)


print(input_value)


iso_onnx_model = onnx.load(path)


onnx.checker.check_model(iso_onnx_model)

[[[0.52143    0.6754695  0.41964117 0.9578651  0.37035188 0.72855943
   0.88130456 0.8519551  0.9605352  0.50641245 0.8341554  0.9424831
   0.35195088 0.64017296 0.2570884  0.99978507 0.12891899 0.8730271
   0.8458719  0.81154907 0.18547893 0.58616465 0.14195919 0.8828586
   0.3138246  0.78482175 0.03386695 0.3036387  0.7117337  0.03048352
   0.61339873 0.6181703  0.2888818  0.5959538  0.09700552 0.44509062
   0.20325193 0.40275708 0.98671865 0.98274964 0.5994578  0.19564176
   0.7526891  0.17145684 0.1092082  0.38063717 0.5230986  0.04728267
   0.02482958 0.98073435 0.2668768  0.29970753 0.3141742  0.12307824
   0.550661   0.92636734 0.7844738  0.526485   0.5489372  0.6372041
   0.18109478 0.8840728  0.64522266 0.5828668  0.6410241  0.4837567
   0.08469291 0.11422633 0.6497292  0.43048826 0.6362229  0.40330258
   0.2624606  0.72204214 0.9611474  0.04919995 0.5424724  0.15422608
   0.03129711 0.10222854 0.04689892 0.35552424 0.9435134  0.12218878
   0.30117017 0.10997052 0.23401923 0.03709656 0.38395905 0.9898242
   0.7141942  0.4955998  0.78369415 0.22568876 0.7187468  0.6965207
   0.47010502 0.50163895 0.3338056 ]]]

In [50]:

session = ort.InferenceSession(iso_onnx_model.SerializeToString())

output1=session.run(None,{inputs:input_value})

print(f"Output: {output1}\n")

Output: [array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)]

In [51]:
graph=gs.import_onnx(iso_onnx_model)

In [60]:
for node in graph.nodes:
    if node.op == "Where":
        print(f"Replacing Where node: {node.name}")

        condition,X, Y = node.inputs
        print(f"condition {condition}")
        print(f"X {X}")
        print(f"Y {Y}")

        output = node.outputs[0]
        
        print(f"Output: {output}")
        

Replacing Where node: /planning_decoder/decoder_blocks.0/r2r_attn/Where

condition Variable (/planning_decoder/decoder_blocks.0/Cast_1_output_0): (shape=[6, 1], dtype=bool)

X Constant (/encoder_blocks.0/attn/Constant_output_0): (shape=[], dtype=float32)

Y Constant (/planning_decoder/decoder_blocks.0/r2r_attn/Constant_1_output_0): (shape=[6, 1], dtype=float32)

Output: Variable (/planning_decoder/decoder_blocks.0/r2r_attn/Where_output_0): (shape=[6, 1], dtype=float32)

In [61]:


# 1. Convert condition to float for element-wise operations
condition_float = gs.Node("Cast", "cast_cond", inputs=[condition], outputs=[gs.Variable("cond_float")], attrs={"to": onnx.TensorProto.FLOAT})


In [ ]:
graph.nodes

In [ ]:
# 2. Compute condition * X
mul_x = gs.Node("Mul", "mul_x", inputs=[condition_float.outputs[0], X], outputs=[gs.Variable("cond_x")])

# 3. Compute (1 - condition) * Y
one_const = gs.Constant("one", np.array(1, dtype=np.float32))
inv_cond = gs.Node("Sub", "inv_cond", inputs=[one_const, condition_float.outputs[0]], outputs=[gs.Variable("inv_cond")])
mul_y = gs.Node("Mul", "mul_y", inputs=[inv_cond.outputs[0], Y], outputs=[gs.Variable("inv_cond_y")])

# 4. Compute output = (condition * X) + ((1 - condition) * Y)
add_node = gs.Node("Add", "add_result", inputs=[mul_x.outputs[0], mul_y.outputs[0]], outputs=[output])

# Replace with new nodes
new_nodes.extend([condition_float, mul_x, inv_cond, mul_y, add_node])
else:
new_nodes.append(node)

# Update the graph with modified nodes
graph.nodes = new_nodes


# Save modified model
onnx.save(gs.export_onnx(graph), output_path)
print(f"Where node replaced and model saved as {output_path}")

